In [1]:
# @title 1. 下載套件
!pip install -q rich
!pip install -q pyngrok
!pip install -q line-bot-sdk
!pip install -q langchain-groq
!pip install langchain-core
!pip install tenacity

# 安裝所需的 Groq 函式庫
!pip install -q groq

# 注意：如果您的 Colab 環境有 GPU，可以將 faiss-cpu 改為 faiss-gpu 以獲得更好效能
!pip install -q langchain langchain-community langchain-huggingface faiss-cpu sentence-transformers
print("必要的函式庫包含已安裝完畢。")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [2]:
# @title 2. 導入需要的工具
import os
from google.colab import userdata
from rich.pretty import pprint
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#新增images資料夾

In [ ]:
# #刪除images的圖片
# import shutil
# import os

# image_folder = 'images'

# if os.path.exists(image_folder):
#     try:
#         shutil.rmtree(image_folder)
#         print(f"資料夾 '{image_folder}' 及其內容已成功刪除。")
#     except OSError as e:
#         print(f"刪除資料夾 '{image_folder}' 時發生錯誤：{e}")
# else:
#     print(f"資料夾 '{image_folder}' 不存在，無需刪除。")

資料夾 'images' 及其內容已成功刪除。


In [ ]:
# #將圖片下載到雲端硬碟
# !cp -r images /content/drive/MyDrive/

In [4]:
# @title 4. 匯入、設定金鑰、常數與模型初始化
import os # 保留 os 模組，以防未來其他地方用到
from groq import Groq
from langchain_groq import ChatGroq
# 確保匯入測試區塊可能需要的模組
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
# 匯入 Colab Secrets 功能
from google.colab import userdata

# --- API 金鑰設定 (使用 Colab Secrets) ---
GROQ_API_KEY = None # 先初始化為 None
try:
    # 從 Colab Secrets 讀取名為 'GROQ_API_KEY' 的密鑰
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')

    if not GROQ_API_KEY:
        # 如果密鑰不存在、值為空或 Notebook access 未啟用，userdata.get 會返回 None
         print("⚠️ 警告：未能從 Colab Secrets 獲取 GROQ_API_KEY。")
         print("   請檢查以下設定：")
         print("   1. 在 Colab 左側的鑰匙(🔑)圖示面板中，是否已建立名稱完全為 'GROQ_API_KEY' 的 Secret？")
         print("   2. 該 Secret 的『值』(Value) 欄位是否已填入您的 Groq API Key？")
         print("   3. 該 Secret 的 'Notebook access' (允許這個筆記本存取) 開關是否已啟用 (ON)？")
         # 此處不拋出錯誤，讓後續的檢查來處理 GROQ_API_KEY 為 None 的情況

except Exception as e:
    # 處理讀取 Secret 過程中的其他潛在錯誤 (雖然較少見)
    print(f"讀取 Colab Secret 'GROQ_API_KEY' 時發生非預期錯誤：{e}")
    GROQ_API_KEY = None # 確保出錯時變數為 None

# --- 常數設定 ---
LLM_MODEL_NAME = "llama3-70b-8192"  # 或是您要使用的其他模型

# --- 初始化 Groq Client ---
client = None
if GROQ_API_KEY: # 只有在成功獲取 API Key 後才嘗試初始化
    try:
        # 使用獲取到的 API Key 初始化 Groq client
        client = Groq(api_key=GROQ_API_KEY)
        print("✅ Groq client 初始化成功。")
    except Exception as e:
        print(f"❌ 初始化 Groq client 時發生錯誤：{e}")
        print("   請檢查您的 API Key 格式是否正確，或是否有網路連線問題。")
        client = None # 確保初始化失敗時 client 為 None
else:
    print("❌ 由於未能獲取 GROQ_API_KEY，無法嘗試初始化 Groq client。")

# --- 初始化 Langchain 的 ChatGroq 模型 ---
llm = None
# 只有在 Groq client 成功初始化後才進行 (這也間接確認了 API Key 存在且初步有效)
if client:
    try:
        llm = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=GROQ_API_KEY)
        print("✅ Langchain ChatGroq 模型初始化成功。")
    except Exception as e_llm:
        print(f"❌ 初始化 Langchain ChatGroq 模型時發生錯誤：{e_llm}")
        llm = None # 確保初始化失敗時 llm 為 None
elif not GROQ_API_KEY:
     print("❌ 由於未獲取 API Key，Langchain ChatGroq 模型無法初始化。")
else: # API Key 存在，但 client 初始化失敗
     print("❌ 由於 Groq client 初始化失敗，Langchain ChatGroq 模型無法初始化。")


# --- 除錯訊息 ---
print("\n--- 初始化狀態 ---")
print(f"GROQ_API_KEY 設定: {'是' if GROQ_API_KEY else '否'}") # 現在反映是否成功從 Secrets 取得
print(f"Groq client 初始化: {'成功' if client else '失敗'}")
print(f"LLM 模型初始化: {'成功' if llm else '失敗'}")
print(f"使用的模型名稱: {LLM_MODEL_NAME}")
print("------------------\n")

# --- 範例測試 (可選) ---
if client and llm:
    print("執行 API 連線與 Langchain 整合測試...")
    try:
        # 測試 Groq API
        response = client.chat.completions.create(
            model=LLM_MODEL_NAME,
            messages=[{"role": "user", "content": "你好！簡單回答即可。"}]
        )
        print("   - Groq API 連線測試成功。")
        # print(response.choices[0].message.content) # 可取消註解查看回應

        # 測試 Langchain 整合
        prompt_template = ChatPromptTemplate.from_template("用一句話總結 Langchain。")
        chain = prompt_template | llm | StrOutputParser()
        langchain_response = chain.invoke({})
        print("   - Langchain 整合測試成功。")
        # print(langchain_response) # 可取消註解查看回應

    except Exception as test_error:
        print(f"\n--- 測試失敗 ---")
        print(f"   測試過程中發生錯誤：{test_error}")
        print("   請檢查您的 API 金鑰有效性、網路連線，以及模型名稱是否正確。")
        print("------------------\n")
else:
    print("\n--- 由於 client 或 llm 未成功初始化，跳過測試。 ---")
    print("---------------\n")

✅ Groq client 初始化成功。
✅ Langchain ChatGroq 模型初始化成功。

--- 初始化狀態 ---
GROQ_API_KEY 設定: 是
Groq client 初始化: 成功
LLM 模型初始化: 成功
使用的模型名稱: llama3-70b-8192
------------------

執行 API 連線與 Langchain 整合測試...
   - Groq API 連線測試成功。
   - Langchain 整合測試成功。


In [8]:
import pandas as pd
import os
import re

try:
    from google.colab import userdata
    print("   - google.colab userdata 模組載入。")
except ImportError:
    print("   - 未在 Colab 環境，將嘗試使用環境變數或預設值（如果適用）。")
    userdata = None

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- Configuration Constants ---
CSV_FILENAME = "GBOM_product_data_name_20250522.csv"
# IMPORTANT: Please confirm these column names based on your CSV.
COLUMN_MATERIAL_ID = "產品編號"        # Assumed Material ID column
COLUMN_MATERIAL_NAME = "產品名稱" # Assumed Material Name column
LLM_MODEL_NAME = "llama3-70b-8192"    # Or your preferred Groq model

# --- Global LLM variable ---
llm = None

def initialize_llm():
    """Initializes the Groq LLM."""
    global llm
    print(f"1. 初始化 LLM ({LLM_MODEL_NAME})...")
    groq_api_key = None
    try:
        if userdata:
            groq_api_key = userdata.get('GROQ_API_KEY')
        else:
            groq_api_key = os.environ.get('GROQ_API_KEY')

        if not groq_api_key:
            print("❌ 錯誤：缺少 GROQ API 金鑰。請在 Colab Secrets 或環境變數中設定 'GROQ_API_KEY'。")
            return False

        llm = ChatGroq(model=LLM_MODEL_NAME, groq_api_key=groq_api_key, temperature=0.1)
        print(f"   ✅ LLM ({LLM_MODEL_NAME}) 初始化成功。")
        return True
    except Exception as e:
        print(f"❌ 初始化 LLM 時發生錯誤: {e}")
        return False

def load_and_filter_f_materials(csv_filepath, id_column, name_column):
    """Loads the CSV, filters for 'G' prefixed Material IDs, and returns their names."""
    print(f"\n2. 讀取並篩選 CSV 檔案: {csv_filepath}")
    try:
        df = pd.read_csv(csv_filepath)
        print(f"   ✅ CSV 讀取成功，共 {len(df)} 行。")
    except FileNotFoundError:
        print(f"❌ 錯誤: CSV 檔案 '{csv_filepath}' 找不到。")
        return None
    except Exception as e:
        print(f"❌ 讀取 CSV 時發生錯誤: {e}")
        return None

    if id_column not in df.columns:
        print(f"❌ 錯誤: CSV 中找不到「原料編號」欄位 '{id_column}'。可用欄位: {df.columns.tolist()}")
        return None
    if name_column not in df.columns:
        print(f"❌ 錯誤: CSV 中找不到「原料名稱」欄位 '{name_column}'。可用欄位: {df.columns.tolist()}")
        return None

    # Ensure Material ID column is treated as string for filtering
    df[id_column] = df[id_column].astype(str)

    # Filter for Material IDs starting with 'F' (case-insensitive)
    f_materials_df = df[df[id_column].str.upper().str.startswith('G')]

    if f_materials_df.empty:
        print(f"   ℹ️ 在 '{id_column}' 欄位中未找到任何以 'G' 開頭的原料編號。")
        return []

    material_names_list = f_materials_df[name_column].dropna().unique().tolist()
    print(f"   ✅ 找到 {len(material_names_list)} 個以 'G' 開頭的獨立原料名稱。")
    return material_names_list

def get_related_materials_with_llm(search_term, f_material_names_list):
    """Uses LLM to find material names related to the search_term from the provided list."""
    global llm
    if not llm:
        print("❌ LLM 未初始化。")
        return ["LLM 未初始化，無法查詢。"]

    if not f_material_names_list:
        return ["沒有以 'G' 開頭的原料名稱可供查詢。"]

    print(f"\n3. 使用 LLM 從 'G' 開頭的原料列表中尋找與 '{search_term}' 相關的項目...")

    material_list_str = "\n".join(f"- {name}" for name in f_material_names_list)

    prompt_template = ChatPromptTemplate.from_template(
        """您的任務是作為一個精確的項目篩選器。
我會提供一個「搜尋關鍵字」和一個「原料名稱列表」。
請您仔細檢查「原料名稱列表」，並從中選出所有與「搜尋關鍵字」直接相關或屬於其類別的「原料名稱」。

搜尋關鍵字: "{search_term}"

原料名稱列表 (請僅從此列表中選取):{material_list_str}

輸出指示:
1.  僅回傳從「原料名稱列表」中選出的、與「搜尋關鍵字」相關的原料名稱。
2.  每個相關的原料名稱各佔一行。
3.  請勿添加任何額外的文字、解釋、編號或引導語句 (例如不要說 "以下是相關的原料：" 或類似的話)。
4.  如果「原料名稱列表」中沒有任何項目與「搜尋關鍵字」相關，請【直接輸出】固定文本："NO_RELATED_MATERIALS_FOUND"。

請開始您的輸出："""
    )

    chain = prompt_template | llm | StrOutputParser()

    try:
        response_str = chain.invoke({
            "search_term": search_term,
            "material_list_str": material_list_str
        })
        print(f"   ℹ️ LLM 原始回應 (前200字): {response_str[:200].replace(os.linesep, ' ')}")

        if "NO_RELATED_MATERIALS_FOUND" in response_str:
            return []

        # Split by newline and remove any empty strings or extra whitespace
        related_names = [name.strip().removeprefix("-").strip() for name in response_str.splitlines() if name.strip()]

        # Optional: Filter to ensure LLM only returned names that were actually in the original list
        # This can help if the LLM hallucinates or slightly alters names.
        original_f_names_set = set(f_material_names_list)
        validated_related_names = [name for name in related_names if name in original_f_names_set and name]

        if not validated_related_names and related_names: # LLM returned something, but none were in original list after validation
            print("   ⚠️ LLM 回應的項目經過驗證後，未發現有效對應原始 G 開頭列表中的項目。可能 LLM 未嚴格遵循指示。")
            # print(f"   LLM返回但未通過驗證的項目: {related_names}")
            # return [] # Or return related_names if you trust LLM's slight variations

        return validated_related_names if validated_related_names else related_names # Prefer validated, fallback to raw LLM split

    except Exception as e:
        print(f"❌ LLM 查詢時發生錯誤: {e}")
        # traceback.print_exc() # For debugging
        return [f"LLM 查詢失敗: {str(e)[:100]}"]

if __name__ == "__main__":
    if not initialize_llm():
        print("由於 LLM 初始化失敗，程式即將結束。")
    else:
        # Load F-materials ONCE before the loop
        print("="*50)
        print("程式啟動：正在預先載入及篩選 'G' 開頭原料資料，請稍候...")
        f_materials_master_list = load_and_filter_f_materials(
            CSV_FILENAME,
            COLUMN_MATERIAL_ID,
            COLUMN_MATERIAL_NAME
        )
        print("="*50)

        if f_materials_master_list is None: # Indicates an error during CSV loading/parsing
            print("因讀取或解析 CSV 檔案失敗，無法啟動查詢迴圈。請檢查檔案和欄位名稱設定。")
        elif not f_materials_master_list: # Indicates CSV loaded, but no F-materials found or their names were empty
            print(f"CSV 檔案已載入，但在 '{COLUMN_MATERIAL_ID}' 欄位未找到以 'F' 開頭的有效原料，或其對應的 '{COLUMN_MATERIAL_NAME}' 為空。")
            print("無法進行後續的 LLM 查詢。請檢查您的 CSV 資料。")
        else:
            print(f"\n✅ 'F' 開頭原料資料載入完成，共找到 {len(f_materials_master_list)} 筆獨立原料名稱。系統已就緒。")

            while True:
                print("-"*50)
                user_search_term = input("\n➡️ 請輸入您要查詢的原料相關詞彙 (或輸入 'exit'/'quit' 離開): ").strip()

                if not user_search_term:
                    print("⚠️ 您沒有輸入查詢詞彙，請重新輸入。")
                    continue

                if user_search_term.lower() in ['exit', 'quit']:
                    print("\n感謝使用，程式即將結束。")
                    break

                print(f"\n您輸入的查詢詞是: '{user_search_term}'")

                # Use the pre-loaded f_materials_master_list
                related_f_materials = get_related_materials_with_llm(user_search_term, f_materials_master_list)

                print("\n--- 查詢結果 ---")
                if related_f_materials: # Check if the list itself is not empty
                    # Check if the first item suggests an error message was returned
                    is_error_message = isinstance(related_f_materials, list) and \
                                       len(related_f_materials) == 1 and \
                                       isinstance(related_f_materials[0], str) and \
                                       ("LLM" in related_f_materials[0] or "錯誤" in related_f_materials[0] or "失敗" in related_f_materials[0])

                    if is_error_message:
                        print(f"❗ 查詢過程中發生問題: {related_f_materials[0]}")
                    elif not any(related_f_materials): # Handles list of empty strings if that somehow occurs
                         print(f"✅ LLM 分析完成，但未在 'F' 開頭的原料中找到與 '{user_search_term}' 明確相關的項目。")
                    else:
                        print(f"與 '{user_search_term}' 相關且原料編號以 'F' 開頭的原料名稱有：")
                        for name in related_f_materials:
                            print(f"- {name}")
                else: # Empty list returned (e.g., NO_RELATED_MATERIALS_FOUND or validated list became empty)
                    print(f"未能在 'G' 開頭的原料中找到與 '{user_search_term}' 相關的項目。")

    print("\n--- 程式執行完畢 ---")

   - google.colab userdata 模組載入。
1. 初始化 LLM (llama3-70b-8192)...
   ✅ LLM (llama3-70b-8192) 初始化成功。
程式啟動：正在預先載入及篩選 'G' 開頭原料資料，請稍候...

2. 讀取並篩選 CSV 檔案: GBOM_product_data_name_20250522.csv
   ✅ CSV 讀取成功，共 116 行。
   ✅ 找到 115 個以 'G' 開頭的獨立原料名稱。

✅ 'F' 開頭原料資料載入完成，共找到 115 筆獨立原料名稱。系統已就緒。
--------------------------------------------------

➡️ 請輸入您要查詢的原料相關詞彙 (或輸入 'exit'/'quit' 離開): 可可'

您輸入的查詢詞是: '可可''

3. 使用 LLM 從 'G' 開頭的原料列表中尋找與 '可可'' 相關的項目...
   ℹ️ LLM 原始回應 (前200字): 配料-銓綿-麥芽巧克力 配料-銓綿-巧克力粉 配料-銓綿-白巧克力風味粉 配料-安立司-巧克力粉-A 配料-義峰-巧克力奶粉 配料-安中-Deluxe黑巧克力霜淇淋粉 配料-皇品-巧克力霜淇淋粉

--- 查詢結果 ---
與 '可可'' 相關且原料編號以 'F' 開頭的原料名稱有：
- 配料-銓綿-麥芽巧克力
- 配料-銓綿-巧克力粉
- 配料-銓綿-白巧克力風味粉
- 配料-安立司-巧克力粉-A
- 配料-義峰-巧克力奶粉
- 配料-安中-Deluxe黑巧克力霜淇淋粉
- 配料-皇品-巧克力霜淇淋粉
--------------------------------------------------

➡️ 請輸入您要查詢的原料相關詞彙 (或輸入 'exit'/'quit' 離開): 巧克力布丁

您輸入的查詢詞是: '巧克力布丁'

3. 使用 LLM 從 'G' 開頭的原料列表中尋找與 '巧克力布丁' 相關的項目...
   ℹ️ LLM 原始回應 (前200字): 配料-銓綿-麥芽巧克力 配料-銓綿-巧克力粉 配料-銓綿-白巧克力風味粉 配料-安立司-巧克力粉-A 配料-義峰-巧克力奶粉 配料-安中-Deluxe黑巧克力霜淇淋粉 配料-皇品-巧克力

KeyboardInterrupt: Interrupted by user

In [ ]:
#md 轉 pdf

In [ ]:
# # @title 安裝 pandoc 和 LaTeX (支援中文)
# print("正在更新套件列表...")
# !apt-get update -qq
# print("正在安裝 pandoc, texlive-xetex 及相關字型...")
# !apt-get install -y pandoc texlive-xetex texlive-fonts-recommended texlive-plain-generic fonts-noto-cjk -qq
# print("安裝完成！")

正在更新套件列表...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
正在安裝 pandoc, texlive-xetex 及相關字型...
Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package fonts-droid-fallback.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../00-fonts-droid-fallback_1%3a6.0.1r16-1.1build1_all.deb ...
Unpacking fonts-droid-fallback (1:6.0.1r16-1.1build1) ...
Selecting previously unselected package fonts-lato.
Preparing to unpack .../01-fonts-lato_2.0-2.1_all.deb ...
Unpacking fonts-lato (2.0-2.1) ...
Selecting previously unselected package poppler-data.
Preparing to unpack .../02-poppler-data_0.4.11-1_all.deb ...
Unpacking poppler-data (0.4.11-1) ...
Selecting previously unselected package tex-common.
Preparing to unpack .../03-tex-common_6.17_all.deb ...
Unpacking tex

In [ ]:
# # @title 使用 pandoc 將 Markdown 轉換為 PDF

# import os # 匯入 os 模組

# # --- 設定 ---
# markdown_input_file = "simplified_output_by_section.md" # 您的簡化後 Markdown 檔案
# pdf_output_file = "simplified_output.pdf"             # 您想輸出的 PDF 檔案名稱

# # --- 檢查輸入檔案是否存在 ---
# if not os.path.exists(markdown_input_file):
#     print(f"錯誤：找不到輸入檔案 '{markdown_input_file}'。請確認檔案存在於 Colab 環境中。")
# else:
#     print(f"找到輸入檔案 '{markdown_input_file}'，開始轉換...")
#     # --- 執行 pandoc 指令 ---
#     # --pdf-engine=xelatex: 使用支援 Unicode (中文) 的 XeLaTeX 引擎
#     # -V mainfont="Noto Sans CJK TC": 指定支援繁體中文的主字型 (Noto Sans CJK TC 是 Colab 通常有的)
#     # 您也可以嘗試 "Noto Serif CJK TC"
#     # -o: 指定輸出檔名
#     !pandoc "{markdown_input_file}" -o "{pdf_output_file}" --pdf-engine=xelatex -V mainfont="Noto Sans CJK TC"

#     # --- 檢查輸出檔案是否成功建立 ---
#     if os.path.exists(pdf_output_file):
#         print(f"\n轉換成功！PDF 檔案已儲存為 '{pdf_output_file}'。")
#         print("您現在可以從 Colab 的檔案瀏覽器下載此 PDF 檔案。")
#         # 可選：自動觸發下載
#         # from google.colab import files
#         # files.download(pdf_output_file)
#     else:
#         print("\n錯誤：轉換失敗，未能生成 PDF 檔案。請檢查上方的 pandoc 輸出是否有錯誤訊息。")

找到輸入檔案 'simplified_output_by_section.md'，開始轉換...

轉換成功！PDF 檔案已儲存為 'simplified_output.pdf'。
您現在可以從 Colab 的檔案瀏覽器下載此 PDF 檔案。


In [ ]:
# # @title 下載 PDF 檔案

# from google.colab import files
# import os

# pdf_filename = "simplified_output.pdf" # 確認這是您生成的 PDF 檔名

# # 檢查檔案是否存在
# if os.path.exists(pdf_filename):
#     print(f"準備下載檔案: {pdf_filename}")
#     # 觸發瀏覽器下載
#     files.download(pdf_filename)
# else:
#     print(f"錯誤：在 Colab 中找不到檔案 '{pdf_filename}'，無法下載。")
#     print("請確認上一步的 pandoc 轉換是否成功生成了此檔案。")

準備下載檔案: simplified_output.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>